In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

CREDENTIALS = service_account.Credentials.from_service_account_file('/home/felipe/inep-data/key/inep-286513-1145bd3494b1.json')
PROJECT_ID = 'inep-286513'
client = bigquery.Client(project=PROJECT_ID, credentials=CREDENTIALS)

subset = 'censo_escolar'
filename = '2019_ESCOLAS.CSV'

table_id = f"{PROJECT_ID}.{subset}.{filename.split('.')[0]}"

job_config = bigquery.LoadJobConfig(
    autodetect=True, source_format=bigquery.SourceFormat.CSV,
    allow_jagged_rows = True,
    encoding='ISO-8859-1')

uri = f"gs://{PROJECT_ID}-{subset}/{filename}"

In [2]:
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

BadRequest: 400 Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 11361; errors: 1. Please look into the errors[] collection for more details.

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ies').getOrCreate()

path = '/home/felipe/inep-data/data/unzipped/censo_escolar/microdados_educacao_basica_2019/microdados_educacao_basica_2019/DADOS/2019_ESCOLAS.CSV'

df = spark.read.csv(path,
                    inferSchema = True,
                    header = True,
                    encoding='ISO-8859-1',
                    sep = '|')